In [2]:
#import adafruit_dht
import argparse
import numpy as np
import time
import tensorflow as tf
#from board import D4



In [3]:
tensor_specs = (tf.TensorSpec([None, 6, 2], dtype=tf.float32),
tf.TensorSpec([None, 2]))
train_ds = tf.data.experimental.load('I:/Polito/ML4IOT/CODES TO DO/Lab3/ex1/th_train', tensor_specs)
val_ds = tf.data.experimental.load('I:/Polito/ML4IOT/CODES TO DO/Lab3/ex1/th_val', tensor_specs)
test_ds = tf.data.experimental.load('I:/Polito/ML4IOT/CODES TO DO/Lab3/ex1/th_test', tensor_specs)

In [4]:
#it = iter(train_ds)
#it.get_next()

In [5]:
converter = tf.lite.TFLiteConverter.from_saved_model('I:\Polito\ML4IOT\CODES TO DO\Lab3\ex1\MLP')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
tflite_model_dir = 'I:\Polito\ML4IOT\CODES TO DO\Lab3\MLP.tflite'
with open(tflite_model_dir, 'wb') as fp:
    fp.write(tflite_model)

In [6]:
interpreter = tf.lite.Interpreter(model_path='I:\Polito\ML4IOT\CODES TO DO\Lab3\MLP.tflite')
interpreter.allocate_tensors()

In [7]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [8]:
input_shape = input_details[0]['shape']

In [9]:
window = np.zeros([1, 6, 2], dtype=np.float32)
expected = np.zeros(2, dtype=np.float32)

MEAN = np.array([9.107597, 75.904076], dtype=np.float32)
STD = np.array([ 8.654227, 16.557089], dtype=np.float32)

In [10]:
def calculateMAE(A, B):
    n = len(A)
    sum = 0
    
        # for loop for iteration
    for i in range(n):
        sum += np.absolute(A[i] - B[i])
    
    mae = sum/n
    return mae

In [11]:
co = 0
measured_list = []
predicted_list = []
for input_data in test_ds.as_numpy_iterator():
    co = co+1
    if co==10:
        break
    for i in range(7):
            #input_data = np.array(np.random.random_sample(([1, 7, 2])),dtype=np.float32)
            #temperature = temp_hum_setc
            #humidity = temp_hum_set[0][i][1]
            if i < 6:
                window[0, i, 0] = np.float32(input_data[0][0][i][0])
                window[0, i, 1] = np.float32(input_data[0][0][i][1])
            if i == 6:
                expected[0] = np.float32(input_data[1][0][0])
                expected[1] = np.float32(input_data[1][0][1])

                #window = (window - MEAN) / STD
                interpreter.set_tensor(input_details[0]['index'], window)
                interpreter.invoke()
                predicted = interpreter.get_tensor(output_details[0]['index'])

               # print('Measured: {:.1f},{:.1f}'.format(expected[0], expected[1]))
                measured_list.append(expected[0])
                measured_list.append(expected[1])

                #print('Predicted: {:.1f},{:.1f}'.format(predicted[0, 0],
                 #   predicted[0, 1]))
                predicted_list.append(predicted[0, 0])
                predicted_list.append(predicted[0, 1])

calculateMAE(measured_list,predicted_list)


1.4139900936020746

In [12]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()